In [1]:
import pyranges as pr
import seaborn as sns
import cerberus

In [39]:
df = pr.read_gtf('45_EU5_GM12878.isoforms.gtf').df
gtf_df = pr.read_gtf('45_EU5_GM12878.isoforms.gtf').df

In [38]:
ref = '/Users/fairliereese/Documents/programming/mortazavi_lab/ref/gencode.v40/gencode.v40.annotation.gtf'
ref_df = pr.read_gtf(ref).df

In [40]:
df = df[['gene_id', 'transcript_id', 'Strand']].drop_duplicates()

In [41]:
df['gid_stable'] = cerberus.get_stable_gid(df, 'gene_id')

In [42]:
ref_df = ref_df[['gene_id', 'Strand']].drop_duplicates()

In [44]:
ref_df.head()

,gene_id,Strand,gid_stable
0,ENSG00000223972.5,+,ENSG00000223972
12,ENSG00000243485.5,+,ENSG00000243485
20,ENSG00000284332.1,+,ENSG00000284332
23,ENSG00000268020.3,+,ENSG00000268020
26,ENSG00000240361.2,+,ENSG00000240361


In [45]:
ref_df = ref_df[['gid_stable', 'Strand']]
ref_df.head()

,gid_stable,Strand
0,ENSG00000223972,+
12,ENSG00000243485,+
20,ENSG00000284332,+
23,ENSG00000268020,+
26,ENSG00000240361,+


In [46]:
# merge w/ reference to figure out which transcripts are antisense
df = df.merge(ref_df,
              how='left',
              on='gid_stable',
              suffixes=('', '_ref'))

In [47]:
df.loc[df.transcript_id=='8e8ddb4c-21fc-48ce-86a7-3efd6499d489:0-0']

,gene_id,transcript_id,Strand,gid_stable,Strand_ref
2778,ENSG00000290127.2,8e8ddb4c-21fc-48ce-86a7-3efd6499d489:0-0,-,ENSG00000290127,NaN


In [23]:
df = df.loc[df.Strand!=df.Strand_ref]

In [24]:
df.head()

,gene_id,transcript_id,Strand,gid_stable,Strand_ref
577,ENSG00000288859.1,4e4111a9-9f7a-4526-9b30-75ae187cf1d6:0-0,-,ENSG00000288859,+
2316,ENSG00000272654.3,624ce7de-b726-4ba0-a4a1-960b8b8d46bf:0-0,-,ENSG00000272654,+
3263,ENSG00000179833.4,62d10d43-45ed-4e77-aa80-3d2e26bb0c95:0-0,+,ENSG00000179833,-
7163,ENSG00000279799.1,e3b610c8-2d47-4071-969d-1e821c925426:0-0,-,ENSG00000279799,+
7316,ENSG00000277075.4,c313ce47-4f03-4a54-a966-6709cb4d1c00:0-0,-,ENSG00000277075,+


In [26]:
tids = df.transcript_id.unique().tolist()
inds = gtf_df.loc[gtf_df.transcript_id.isin(tids)].index
gtf_df.loc[inds, 'gene_id'] = 'novel_gene_antisense_'+gtf_df.loc[inds, 'gene_id']


In [32]:
gtf_df.loc[inds].head()

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,transcript_id,exon_number
14943,chr1,FLAIR,transcript,149851060,149851863,.,-,.,novel_gene_antisense_ENSG00000288859.1,4e4111a9-9f7a-4526-9b30-75ae187cf1d6:0-0,NaN
14944,chr1,FLAIR,exon,149851060,149851863,.,-,.,novel_gene_antisense_ENSG00000288859.1,4e4111a9-9f7a-4526-9b30-75ae187cf1d6:0-0,0
14985,chr1,FLAIR,transcript,153977682,153978908,.,-,.,novel_gene_antisense_ENSG00000272654.3,624ce7de-b726-4ba0-a4a1-960b8b8d46bf:0-0,NaN
14986,chr1,FLAIR,exon,153977682,153978908,.,-,.,novel_gene_antisense_ENSG00000272654.3,624ce7de-b726-4ba0-a4a1-960b8b8d46bf:0-0,0
23620,chr2,FLAIR,transcript,64750577,64750895,.,+,.,novel_gene_antisense_ENSG00000179833.4,62d10d43-45ed-4e77-aa80-3d2e26bb0c95:0-0,NaN


In [27]:
len(gtf_df.gene_id.unique())

18274

In [28]:
len(gtf_df[['Strand', 'gene_id']].drop_duplicates())

18420

In [33]:
temp = gtf_df[['Strand', 'gene_id']].drop_duplicates()
temp.loc[temp.gene_id.duplicated(keep=False)].sort_values(by='gene_id').head()

,Strand,gene_id
3609,+,ENSG00000290127.2
15155,-,ENSG00000290127.2
11764,-,ENSG00000291233.2
4705,+,ENSG00000291233.2
90963,+,chr10:101060000


In [35]:
gtf_df.loc[gtf_df.gene_id=='ENSG00000290127.2']

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,transcript_id,exon_number
3609,chr1,FLAIR,transcript,197201480,197202322,.,+,.,ENSG00000290127.2,ENST00000843833.1,NaN
3610,chr1,FLAIR,exon,197201480,197201588,.,+,.,ENSG00000290127.2,ENST00000843833.1,0
3611,chr1,FLAIR,exon,197202188,197202322,.,+,.,ENSG00000290127.2,ENST00000843833.1,1
15155,chr1,FLAIR,transcript,197201505,197202322,.,-,.,ENSG00000290127.2,8e8ddb4c-21fc-48ce-86a7-3efd6499d489:0-0,NaN
15156,chr1,FLAIR,exon,197201505,197202322,.,-,.,ENSG00000290127.2,8e8ddb4c-21fc-48ce-86a7-3efd6499d489:0-0,0


In [36]:
tids

['4e4111a9-9f7a-4526-9b30-75ae187cf1d6:0-0',
 '624ce7de-b726-4ba0-a4a1-960b8b8d46bf:0-0',
 '62d10d43-45ed-4e77-aa80-3d2e26bb0c95:0-0',
 'e3b610c8-2d47-4071-969d-1e821c925426:0-0',
 'c313ce47-4f03-4a54-a966-6709cb4d1c00:0-0',
 '8c2bd8f7-d7d9-4144-87f0-eda0fdd3ec90:0-0',
 '69589786-8c41-404d-bd01-ecce4bbcd87b:0-0',
 '08088ba1-a76e-4618-a30d-0533e8c69c3c:0-0',
 '7c476f81-7d3f-41f6-bb6f-81cebf6efd23:0-0',
 '439aedc2-2cad-4bd3-b3e8-e72e3dbedafd:0-0',
 'f191e4cc-9f56-4cd0-80f2-7abece82afc8:0_2-0',
 '6f3ec174-c2a3-4af4-be56-3b7ebf6713c5:0-0',
 '5a2949ff-3248-4da5-91af-c2a968e9df51:0-0',
 'bbc3d1c8-e4b7-4861-a1df-c1a6a2bb01be:0-0']